# Importación de librerías

In [1]:
import spacy

from sense2vec import Sense2Vec

import gensim
import gensim.downloader as gensim_api
from gensim.models.doc2vec import Doc2Vec, TaggedDocument

from datasets import load_dataset

/home/wrstdani/anaconda3/envs/pln1/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


---
# Ejercicio 1
- Una de las principales limitaciones presentes en la implementación original de Word2Vec es la asignación de un único vector estático a cada palabra. Además de los posibles problemas relacionados con las palabras fuera del vocabulario de entrenamiento, este hecho puede suponer un gran inconveniente a la hora de analizar palabras polisémicas con diversas categorías gramaticales, no pudiendo diferenciar sus posibles acepciones.

- En este contexto, Sense2Vec surge como una posible solución a este problema. En lugar de tratar cada token como una entidad única, Sense2Vec propone realizar un etiquetado previo de los tokens de entrenamiento con su correspondiente etiqueta gramatical o etiqueta POS. Por medio de la incorporación de esta información lingüística, Sense2Vec permite generar representaciones diferenciadas de palabras según su categoría gramatical, facilitando el análisis de palabras polisémicas.

In [2]:
"""
!mkdir models && \
    wget -P  https://github.com/explosion/sense2vec/releases/download/v1.0.0/s2v_reddit_2015_md.tar.gz && \
    tar -xvzf models/s2v_reddit_2015_md.tar.gz \
    rm models/s2v_reddit_2015_md.tar.gz
"""

'\n!mkdir models &&     wget -P  https://github.com/explosion/sense2vec/releases/download/v1.0.0/s2v_reddit_2015_md.tar.gz &&     tar -xvzf models/s2v_reddit_2015_md.tar.gz     rm models/s2v_reddit_2015_md.tar.gz\n'

In [3]:
s2v = Sense2Vec().from_disk("models/s2v_old")

- Se pide:
    - Calcular con Sense2Vec los embeddings de la palabra "watch" tanto cuando ejerce la categoría gramatical de verbo como cuando ejerce la de nombre. ¿Son diferentes los vectores?
    - Dado el conjunto de frases ["You must watch the light movie.", "He gave me a watch with a light."], elimina las palabras vacías y signos de puntuación. Posteriormente, obtén con Sense2Vec las tres palabras más similares de los tokens resultantes, empleando para ello su categoría gramatical.
    - Usando la librería Gensim, repite lo anterior para el modelo preentrenado "word2vec-google-news-300". ¿Qué diferencias encuentras entre Word2Vec y Sense2Vec? ¿Cuál funciona mejor y sobre qué contextos?
    - Con Word2Vec y Sense2Vec, repite lo anterior para el conjunto de frases ["I went to the bank to deposit my money.", "The land along the river bank has vegetation."]. ¿Qué ocurre en esta ocasión? ¿A qué se debe?
    - Calcula la similitud coseno entre los siguientes pares con la categoría gramatical indicada:
        - ("watch|NOUN", "watch|VERB") ¿Es alto este valor? ¿A qué puede deberse?
        - ("watch|NOUN", "clock|NOUN")
        - ("watch|NOUN", "view|VERB")
        - ("watch|VERB", "clock|NOUN")
        - ("watch|VERB", "view|VERB")

## Obtener embeddings con S2V

In [4]:
# Obtener embeddings de "watch"
watch_verb = dict(s2v.items())["watch|VERB"]
watch_noun = dict(s2v.items())["watch|NOUN"]

## Obtener palabras más similares (S2V vs. W2V)

In [5]:
sents = [
    "You must watch a light movie.",
    "He gave me a watch with a light."
]

nlp = spacy.load("en_core_web_sm")
tok_tagged_sents = [[(token.lower_, token.pos_) for token in nlp(s) if (token.is_alpha and (not token.is_stop))] for s in sents]

print(f"- Sentencias tokenizadas y etiquetadas (POS tagged):\n{tok_tagged_sents}")

print("- 3 palabras más similares de...")
for s in tok_tagged_sents:
    for word, pos_tag in s:
        print(f"    - ({word}|{pos_tag}):")
        three_most_similar = s2v.most_similar("|".join([word, pos_tag]), n=3)
        for ms in three_most_similar:
            w = ms[0].split("|")[0]
            pt = ms[0].split("|")[1]
            sim = ms[1]
            print(f"        - {w} siendo un {pt} tiene un {sim*100:.2f}% de similitud")

- Sentencias tokenizadas y etiquetadas (POS tagged):
[[('watch', 'VERB'), ('light', 'ADJ'), ('movie', 'NOUN')], [('gave', 'VERB'), ('watch', 'NOUN'), ('light', 'NOUN')]]
- 3 palabras más similares de...
    - (watch|VERB):
        - watching siendo un VERB tiene un 89.33% de similitud
        - watching siendo un NOUN tiene un 83.48% de similitud
        - watched siendo un VERB tiene un 82.30% de similitud
    - (light|ADJ):
        - dark siendo un ADJ tiene un 80.24% de similitud
        - bright siendo un ADJ tiene un 79.52% de similitud
        - light siendo un NOUN tiene un 76.95% de similitud
    - (movie|NOUN):
        - whole_movie siendo un NOUN tiene un 90.32% de similitud
        - good_movie siendo un NOUN tiene un 90.19% de similitud
        - terrible_movie siendo un NOUN tiene un 89.95% de similitud
    - (gave|VERB):
        - Gave siendo un VERB tiene un 84.35% de similitud
        - giving siendo un VERB tiene un 80.08% de similitud
        - had siendo un VERB tien

In [6]:
# Utilizar Word2Vec
w2v = gensim_api.load("word2vec-google-news-300")
print("- 3 palabras más similares de...")
for s in tok_tagged_sents:
    for word, pos_tag in s:
        print(f"    - {word}:")
        three_most_similar = w2v.most_similar(word, topn=3)
        for ms in three_most_similar:
            w = ms[0]
            sim = ms[1]
            print(f"        - {w} tiene un {sim*100:.2f}% de similitud")

- 3 palabras más similares de...
    - watch:
        - watching tiene un 78.36% de similitud
        - watched tiene un 66.77% de similitud
        - Watching tiene un 63.86% de similitud
    - light:
        - lights tiene un 55.06% de similitud
        - yellowish_glow tiene un 54.85% de similitud
        - illumination tiene un 53.43% de similitud
    - movie:
        - film tiene un 86.77% de similitud
        - movies tiene un 80.13% de similitud
        - films tiene un 73.63% de similitud
    - gave:
        - give tiene un 74.12% de similitud
        - giving tiene un 74.06% de similitud
        - gives tiene un 66.45% de similitud
    - watch:
        - watching tiene un 78.36% de similitud
        - watched tiene un 66.77% de similitud
        - Watching tiene un 63.86% de similitud
    - light:
        - lights tiene un 55.06% de similitud
        - yellowish_glow tiene un 54.85% de similitud
        - illumination tiene un 53.43% de similitud


In [7]:
sents = [
    "I went to the bank to deposit my money.",
    "The land along the river bank has vegetation."
]

nlp = spacy.load("en_core_web_sm")
tok_tagged_sents = [[(token.lower_, token.pos_) for token in nlp(s) if (token.is_alpha and (not token.is_stop))] for s in sents]

print(f"- Sentencias tokenizadas y etiquetadas (POS tagged):\n{tok_tagged_sents}")

print("- 3 palabras más similares de...")
for s in tok_tagged_sents:
    for word, pos_tag in s:
        print(f"    - ({word}|{pos_tag}):")
        print("         - Sense2Vec:")
        three_most_similar = s2v.most_similar("|".join([word, pos_tag]), n=3)
        for ms in three_most_similar:
            w = ms[0].split("|")[0]
            pt = ms[0].split("|")[1]
            sim = ms[1]
            print(f"            - {w} siendo un {pt} tiene un {sim*100:.2f}% de similitud")
        
        print("         - Word2Vec:")
        three_most_similar = w2v.most_similar(word, topn=3)
        for ms in three_most_similar:
            w = ms[0]
            sim = ms[1]
            print(f"            - {w} tiene un {sim*100:.2f}% de similitud")

- Sentencias tokenizadas y etiquetadas (POS tagged):
[[('went', 'VERB'), ('bank', 'NOUN'), ('deposit', 'VERB'), ('money', 'NOUN')], [('land', 'NOUN'), ('river', 'NOUN'), ('bank', 'NOUN'), ('vegetation', 'NOUN')]]
- 3 palabras más similares de...
    - (went|VERB):
         - Sense2Vec:
            - went siendo un ADJ tiene un 88.34% de similitud
            - came siendo un VERB tiene un 88.03% de similitud
            - wen't siendo un VERB tiene un 87.92% de similitud
         - Word2Vec:
            - came tiene un 71.42% de similitud
            - ran tiene un 67.15% de similitud
            - gone tiene un 64.05% de similitud
    - (bank|NOUN):
         - Sense2Vec:
            - local_bank siendo un NOUN tiene un 88.59% de similitud
            - bank_account siendo un NOUN tiene un 86.53% de similitud
            - same_bank siendo un NOUN tiene un 85.36% de similitud
         - Word2Vec:
            - banks tiene un 74.41% de similitud
            - banking tiene un 69.02% de 

## Similitud utilizando S2V

In [8]:
print(f"- Similitud entre watch (noun) y watch (verb): {s2v.similarity("watch|NOUN", "watch|VERB")}")
print(f"- Similitud entre watch (noun) y clock (noun): {s2v.similarity("watch|NOUN", "clock|NOUN")}")
print(f"- Similitud entre watch (noun) y view (verb): {s2v.similarity("watch|NOUN", "view|VERB")}")
print(f"- Similitud entre watch (verb) y clock (noun): {s2v.similarity("watch|VERB", "clock|NOUN")}")
print(f"- Similitud entre watch (verb) y view (verb): {s2v.similarity("watch|VERB", "view|VERB")}")

- Similitud entre watch (noun) y watch (verb): 0.479771226644516
- Similitud entre watch (noun) y clock (noun): 0.5055921077728271
- Similitud entre watch (noun) y view (verb): 0.3805285096168518
- Similitud entre watch (verb) y clock (noun): 0.3295963704586029
- Similitud entre watch (verb) y view (verb): 0.4352375566959381


---
# Ejercicio 2
- Los modelos de embedding estáticos anteriormente vistos (Word2Vec, GloVe, FastText y Sense2Vec) generan representaciones a nivel palabra. Sin embargo, a la hora de llevar a cabo diferentes tareas de PLN, podría ser interesante obtener una representación a nivel documento que sea capaz de capturar la semántica global del mismo.

- En este contexto surge Doc2Vec, extensión de Wor2Vec diseñada para representar textos de forma integral. A diferencia de los anteriores modelos pre-entrenados que únicamente generan embeddings a nivel de token, Doc2Vec introduce un vector que representa al documento en su totalidad. Con dicho fin, hace uso de, o bien la técnica Distributed Memory (PV-DM), análoga a CBOW, o bien de la técnica Distributed Bag of Words (PV-DBOW), análoga a Skip-Gram. Específicamente, PV-DM predice una palabra en función de su contexto y del vector del documento, mientras que PV-DBOW omite el contexto y utiliza exclusivamente el vector para predecir las palabras que lo conforman.

- Se pide:
    - Preparar conjunto aleatorio de 10000 resúmenes de Wikipedia en inglés.
    - Utiliza TaggedDocument de la librería Gensim para representar el dataset de entrenamiento. Tras ello, entrena un modelo Doc2Vec sobre el anterior conjunto de datos.
    - Dados los resúmenes de Wikipedia de "Federico García Lorca", "Flag of Europe" y "Super Mario 64", calcula para cada uno sus 10 resúmenes más similares. Analiza los resultados. ¿Están relacionados?
    - Dado un nuevo resumen no visto durante el entrenamiento, calcula su vector y analiza los resúmenes más similares al mismo.
    - Representa gráficamente con PCA y la librería Bokeh un subconjunto de los anteriores resúmenes de Wikipedia.
    - Sobre los apartados anteriores, experimenta con diferentes configuraciones de entrenamiento de Doc2Vec.

## Obtener conjunto de resúmenes

In [9]:
dataset = load_dataset("laion/Wikipedia-Abstract", "English")
subset = dataset["train"].select(range(10000))
texts, titles = subset["Abstract"], subset["Title"]

## Preprocesar conjunto de resúmenes

In [ ]:
def preprocess_text(model, text):
    doc = model(text)
    return [token.lower_ for token in doc if ((not token.is_stop) and token.is_alpha)]

# Aplicar pre-procesamiento a todos los textos del corpus
nlp = spacy.load("en_core_web_sm")
preprocessed_texts = [preprocess_text(nlp, s) for s in texts]

## Obtener los `TaggedDocument`s y entrenar el modelo D2V

In [46]:
tagged_documents = [TaggedDocument(words=preprocessed_texts[i], tags=[titles[i]]) for i in range(len(preprocessed_texts))]
d2v = Doc2Vec(tagged_documents, vector_size=100, window=2, min_count=1, workers=4, epochs=200)

## Obtener 10 resúmenes más similares de...

In [20]:
filtered = dataset["train"].filter(lambda x: x["Title"] == "Flag of Europe" or x["Title"] == "Federico García Lorca" or x["Title"] == "Super Mario 64")

Filter: 100%|██████████| 6575217/6575217 [00:42<00:00, 155810.07 examples/s]


In [40]:
filtered_preprocessed = [(t, preprocess_text(nlp, s)) for s, t in zip(filtered["Abstract"], filtered["Title"])]

In [45]:
print("- Top 10 resúmenes más similares a...")
for title, text in filtered_preprocessed:
    print(f"    - {title}:")
    inferred_vector = d2v.infer_vector(text)
    most_similar = d2v.dv.most_similar([inferred_vector], topn=10)
    for ms in most_similar:
        print(f"        - {ms[0]} tiene un {ms[1]*100:.2f}% de similitud")

- Top 10 resúmenes más similares a...
    - Federico García Lorca:
        - Federico García Lorca tiene un 96.60% de similitud
        - Jorge Aragão tiene un 96.36% de similitud
        - Guy of Bazoches tiene un 96.36% de similitud
        - Myrta Silva tiene un 96.28% de similitud
        - Georges Courteline tiene un 96.20% de similitud
        - Jean Mabillon tiene un 96.20% de similitud
        - Bita Farrahi tiene un 96.12% de similitud
        - Philippe Goddin tiene un 95.97% de similitud
        - Allegra Versace tiene un 95.97% de similitud
        - El Cid tiene un 95.96% de similitud
    - Super Mario 64:
        - Super Mario 64 tiene un 96.07% de similitud
        - Super Mario Land 2: 6 Golden Coins tiene un 95.66% de similitud
        - The Time Warp of Dr. Brain tiene un 93.21% de similitud
        - Music of the Final Fantasy series tiene un 92.21% de similitud
        - List of campaign settings tiene un 91.58% de similitud
        - Irregular chess opening tiene u